In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-3NygnvMpPz86jgfVfZdEOE-u5BDezgW..................."

install libraries

In [ ]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

Step1a - Indexing (Document Ingestion)

In [ ]:

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
video_id = "7RTYa-gCRps"
try:
    ytt_api = YouTubeTranscriptApi()
    fetched_transcript = ytt_api.fetch(video_id)
    transcript = " ".join(chunk.text for chunk in fetched_transcript)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

if you could go back and tell that 28-year-old version of yourself something about relationships or about marriage what would you tell that 28y old version it's not just the wedding right A lot of people when they say I I want to get married I'm looking to settle down are always thinking of just the wedding like it's like it's in the movies you know it's wedding end of movie happily ever after and uh but what marriage is is definitely companionship there is someone with you to witness your every days every little thing that happens in your day it shouldn't go unno someone should witness it it can be something as silly as what did you eat for lunch I sprain my toe today you're a very calm guy dude I'm someone who's diagnosed with ADHD so I have intrusive thoughts sometimes when I'm doing a podcast with people I visualize a train coming and running both of us over and all that's the curse of my head which is why all these random questions are being thrown at you wow okay but I mean I've 

in upper we are getting a right

In [ ]:
fetched_transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='if you could go back and tell that', start=0.04, duration=4.16), FetchedTranscriptSnippet(text='28-year-old version of yourself', start=2.399, duration=3.4), FetchedTranscriptSnippet(text='something about relationships or about', start=4.2, duration=3.32), FetchedTranscriptSnippet(text='marriage what would you tell that 28y', start=5.799, duration=3.92), FetchedTranscriptSnippet(text="old version it's not just the wedding", start=7.52, duration=4.279), FetchedTranscriptSnippet(text='right A lot of people when they say I I', start=9.719, duration=3.401), FetchedTranscriptSnippet(text="want to get married I'm looking to", start=11.799, duration=2.8), FetchedTranscriptSnippet(text='settle down are always thinking of just', start=13.12, duration=2.999), FetchedTranscriptSnippet(text="the wedding like it's like it's in the", start=14.599, duration=4.001), FetchedTranscriptSnippet(text="movies you know it's wedding end of", start=16.

step 1b - indexing(text splitter)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [ ]:
len(chunks)

73

In [ ]:
chunks[10] ## to see a partical 1 chunk

Document(metadata={}, page_content="life-changing it is um and you know till then you think you know uh what love is uh what you feel for your mother what you feel for your father uh or your wife or your friends but trust me there is nothing like when you have a child as in the love you feel for that child is it's like it'll be like that you've never had that feeling in your life and I'm like okay this is what love is more than wife more than anything like I mean I'm sure she feels the same way like what you feel for your baby is like and this is why our parents have been the way they are with us you know uh in worrying all the time or uh and also always forgiving you know no matter what you can do the worst thing and your parents will find a way to forgive you because you're you're a part of them what should I know about my 30s because a lot of my friends it's either everyone's afraid about like all these aging weight gain hair loss or people are afraid about life and fatherhood or th

Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: '0a551219-6cff-452a-9211-4c41c9332a62',
 1: '4496c3ab-aa24-4e1f-b463-8578c8f84d0a',
 2: 'a1351ffe-9759-4ee7-a4fa-612d17feed29',
 3: '28c9003e-da35-46cd-b5d1-588f9c411541',
 4: '9807cb05-5ffd-4bf5-978f-afeb1196c7ab',
 5: '7ab12133-d2e1-42ec-9529-773deb140943',
 6: '139645a0-34c6-4be2-9d52-2c9931657144',
 7: 'dad51ac0-2ebb-428a-a954-c577557e4774',
 8: '9041927e-4898-45d9-9474-fed9068217fe',
 9: '32da02bc-dc01-4e46-9856-162df4de3cfc',
 10: '0a61a03b-8a8a-4ae8-b16b-ebdcd7878734',
 11: '2f734008-4a23-460a-b806-ea80d530a7a0',
 12: '8af24a19-4c71-4c1a-a056-52b574c3fb18',
 13: '8ee6fbc0-a3d2-4a85-a7c8-988f57a7dc89',
 14: '7f04e23a-544d-4237-ad4d-8087411dbe89',
 15: '9de35c12-35b5-4182-b814-421dde23ddb0',
 16: 'f8fee6e6-7c1f-491d-890b-4ccc3c15e510',
 17: '5a6c8331-9f47-4728-810b-978a62e3dd4d',
 18: '98e07355-ae6e-4b3f-a86b-01c93dede9a3',
 19: '850051c7-3300-40c4-8943-a88ec1554aa1',
 20: '9c7dcaab-35ac-49c3-a9ab-1dd7b668d71c',
 21: 'f5e5f4d2-cd1e-4267-9c47-02cd6c007da2',
 22: 'f404e78c-4226-

In [ ]:
vector_store.get_by_ids(['996616ef-8b34-43f7-9a0f-9979a094c8f3']) ## to see any
#particular chunk add that id and see

[Document(id='996616ef-8b34-43f7-9a0f-9979a094c8f3', metadata={}, page_content="listener of TRS tell me what you thought of this particular episode I love doing really chilled out conversations like this once in a while if you're a new listener of the show make sure you check out our entire library of all our film oriented podcasts we'll link them down below keep supporting TRS rir and the team will be back very soon [Music] he [Music]")]

Step 2 - Retrival

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fb27a2fe5d0>, search_kwargs={'k': 4})

In [ ]:
retriever.invoke('how old she is')

[Document(id='2452a75b-cda0-4b15-b2c8-135c8e9c10fa', metadata={}, page_content="her I was like I just see her there something you can't like articulate it exactly I can't articulate I just feel like I could see her as a part of my world as a part of my family which i' never seen with anybody else and how does the relationship evolve over time like the longest I've dated someone is like three and a half four years so I don't yeah yeah up until then the same uh but also she was very young when we got married she was only like 23 or something um so she's grown up over time and uh it evolves in the sense see also it was it was a my 30s was essentially me having met someone uh me having taken this giant leap into this career and then trying to make that that um and kind of find each other and literally I came back from my honeymoon and I went straight to shoot my second film started and they waited only for my wedding to get done so we barely had a courtship uh we met and we were married 6 

step 3 - Augmentation

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = "is the wife of salman's  discussed in this video? if yes then who is she"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='0c4f10a7-1122-4e76-add7-2e2637918456', metadata={}, page_content="and uh but what marriage is is definitely companionship uh is definitely you know having that one you know there was there was a a line of film have you seen Shall We Dance no watch that film it's a good film so she Susan sandon says this line where she's like we do you know why we get married or we have a partner uh is so there is someone with you to witness your everyday like every little thing that happens in your day it shouldn't go noticed someone should witness it it can be something as silly as what did you eat for lunch uh or I sprain my toe today I stub my toe today or something the the most trivial of things that happen to you you're not sharing that every day with your parents or your kids or anything I think whoever is your uh partner on this ride called life is the only witness to to your every day have you cried in front of your wife yeah how many times like I the big moments right like the bi

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"and uh but what marriage is is definitely companionship uh is definitely you know having that one you know there was there was a a line of film have you seen Shall We Dance no watch that film it's a good film so she Susan sandon says this line where she's like we do you know why we get married or we have a partner uh is so there is someone with you to witness your everyday like every little thing that happens in your day it shouldn't go noticed someone should witness it it can be something as silly as what did you eat for lunch uh or I sprain my toe today I stub my toe today or something the the most trivial of things that happen to you you're not sharing that every day with your parents or your kids or anything I think whoever is your uh partner on this ride called life is the only witness to to your every day have you cried in front of your wife yeah how many times like I the big moments right like the big like whenever I I get emotional every time a film exceeds my expectation uh\n

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      and uh but what marriage is is definitely companionship uh is definitely you know having that one you know there was there was a a line of film have you seen Shall We Dance no watch that film it's a good film so she Susan sandon says this line where she's like we do you know why we get married or we have a partner uh is so there is someone with you to witness your everyday like every little thing that happens in your day it shouldn't go noticed someone should witness it it can be something as silly as what did you eat for lunch uh or I sprain my toe today I stub my toe today or something the the most trivial of things that happen to you you're not sharing that every day with your parents or your kids or anything I think whoever is your uh partner on this ride called life is the only witness to to y

step 4 - Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, Salman's wife is discussed in the video, but her name is not mentioned. The conversation focuses on their relationship and how she has witnessed his journey, but no specific details about her identity are provided.


In [ ]:
answer

AIMessage(content="Yes, Salman's wife is discussed in the video, but her name is not mentioned. The conversation focuses on their relationship and how she has witnessed his journey, but no specific details about her identity are provided.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 893, 'total_tokens': 935, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C86Zu6kOBS0bxKQckNFRUA14z3VQx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--23023fd5-10e9-4971-a079-939851ae001c-0', usage_metadata={'input_tokens': 893, 'output_tokens': 42, 'total_tokens': 935, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

here we are running each and every function every time so need to make chain

Building a chain (parallel chain)

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('who is Dulquer Salman')

{'context': "but as with any of our film oriented episodes we've completely unlocked both his mind mind and his heart we've also gotten to his soul so you're going to enjoy this conversation I encourage you to make as many reels and shorts as is possible feel free to use the content all I'll say is that this was a very friendly easygoing conversation between two media Bros enjoy yourselves is dkar Salman on TRS [Music] DQ brother dkar Salman by Hi rir how are you bro I'm very well such a calm classy Vibe you have thank you a burning question for you man sure how do you have such beautiful hair I'm just glad it's still there dude it's gorgeous that's the first thing I noticed about you when I saw you ever like for the first time in life I was like this guy has damn and I still haven't done a Hair Commercial no no none of those products yet I'm like I should be doing this a lot of guys are age are struggling with a lot of image related I know my classmates and they all look at me like yo

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you summarize the video')

"The video features a conversation between two individuals, one of whom is a guest named DQ brother dkar Salman. They discuss various topics, including the complexities of cultural interactions in Mumbai, the guest's hair, and the nature of their podcast. The conversation is light-hearted and introspective, with the host encouraging the audience to engage with the content. They reflect on personal experiences and the importance of connecting with the audience through authenticity. Overall, it's a friendly and engaging dialogue that explores deeper thoughts and emotions."

In [ ]:
result = main_chain.invoke('who is Dulquer Salman')

In Colab: Train & Save Model

In [ ]:
import joblib

# Suppose `model` is your trained ML model
joblib.dump(result, "my_result.pkl")


['my_result.pkl']